In [3]:
#imports
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import preprocessing, neighbors

In [4]:
#OBTER OS DADOS
raw_data = pd.read_csv('Social_Network_Ads.csv',header=0,na_values=['?'])

#REMOVER LINHAS COM VALORES INVÁLIDOS
raw_data=raw_data.dropna()
raw_data.replace(['Male', 'Female'], [0, 1],inplace = True)


#ADICIONA UMA SEGUNDA CAMADA, PARA OS CASOS BENIGNOS
raw_data['N_Purchased']=np.where(raw_data['Purchased']==1, 0, 1)


#DIVIDIR DADOS ENTRE ENTRADA E SAÍDA
raw_input_data = raw_data.iloc[:,1:4]
raw_output_data = raw_data.iloc[:,4:]

In [5]:
#FUNÇÕES DE VISUALIZAÇÃO
def plot_acc(data):
    plt.plot(data.history['acc'])
    plt.plot(data.history['val_acc'])
    plt.title('Precisão do Modelo')
    plt.ylabel('Precisão')
    plt.xlabel('Época')
    plt.legend(['Treinamento', 'Validação'], loc='upper left')
    plt.show()
    
def plot_loss(data):
    plt.plot(data.history['loss'])
    plt.plot(data.history['val_loss'])
    plt.title('Erro Médio Quadrático')
    plt.ylabel('Custo')
    plt.xlabel('Época')
    plt.legend(['Treinamento', 'Validação'], loc='upper left')
    plt.show()
    
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result    

In [21]:
#VALORES PARA VARIAR


PCT_EVAL = 0.1
k = 3


In [22]:
#DIVIDE E NORMALIZA OS DADOS

x_train, x_validate, y_train, y_validate = train_test_split(raw_input_data,raw_output_data, test_size=PCT_EVAL, shuffle= True)

x_train = normalize(x_train)
x_validate = normalize(x_validate)

#transforma os datasets em arrays
x_train = np.array(x_train.values)
y_train = np.array(y_train.values)

x_validate = np.array(x_validate.values)
y_validate = np.array(y_validate.values)

#normaliza os dados


#x_train = tf.keras.utils.normalize(x_train,axis=1)
#x_validate= tf.keras.utils.normalize(x_validate,axis=1)


In [23]:
#DEFINE PLACEHOLDERS PARA O TREINAMENTO
L = len(x_train[0])
x_treinamento = tf.placeholder(shape=[None, L], dtype=tf.float32)
y_treinamento = tf.placeholder(shape=[None, len(y_validate[0])], dtype=tf.float32)
x_teste = tf.placeholder(shape=[None, L], dtype=tf.float32)

#DISTANCIA DE MANHATTAN
distance = tf.reduce_sum(tf.abs(tf.subtract(x_treinamento, tf.expand_dims(x_teste, 1))), axis=2)

#ACHA OS K PONTOS MAIS PRÓXIMOS
_, indices_k = tf.nn.top_k(tf.negative(distance), k=k)
top_k = tf.gather(y_treinamento, indices_k)

soma = tf.reduce_sum(top_k, axis=1)
pred = tf.argmax(soma, axis=1)

In [24]:
#TREINAMENTO
sess = tf.Session()
resultado_pred = sess.run(pred, feed_dict={x_treinamento: x_train,
                               x_teste: x_validate,
                               y_treinamento: y_train})

#AVALIAÇÃO
acc = 0
for pred, i in zip(resultado_pred, y_validate):
    if pred == np.argmax(i):
        acc += 1

print(acc / len(resultado_pred))

0.925


In [25]:
#GERA O VETOR DE RESULTADOS ESTIMADOS E OS RESULTADOS CORRETOS
y_true = np.zeros(len(y_validate))
for i in range(len(y_validate)):
    y_true[i] = np.argmax(y_validate[i]) 
p = 10
print(resultado_pred[0:p])    
print(y_true[0:p])
#1 = N COMPROU
#0 = COMPROU

[0 1 1 0 0 1 0 1 0 0]
[1. 1. 1. 0. 0. 1. 0. 1. 0. 1.]


In [26]:
print(confusion_matrix(y_true, resultado_pred))
#VP, FN//FP ,VN

[[12  0]
 [ 3 25]]
